# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [2]:
import re
import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize,RegexpTokenizer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
import pickle
from sklearn.metrics import roc_auc_score , precision_score, recall_score, f1_score, classification_report,accuracy_score,confusion_matrix,roc_curve, auc

In [4]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table("InsertTableName",con= engine)

In [5]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [6]:
df.isna().sum()

id                            0
message                       0
original                  16046
genre                         0
related                       0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
child_alone                   0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops                         0
aid_centers                   0
other_in

In [7]:
# Note : Original has many na's , so we would omit that column.

In [8]:
df.dtypes

id                         int64
message                   object
original                  object
genre                     object
related                    int64
request                    int64
offer                      int64
aid_related                int64
medical_help               int64
medical_products           int64
search_and_rescue          int64
security                   int64
military                   int64
child_alone                int64
water                      int64
food                       int64
shelter                    int64
clothing                   int64
money                      int64
missing_people             int64
refugees                   int64
death                      int64
other_aid                  int64
infrastructure_related     int64
transport                  int64
buildings                  int64
electricity                int64
tools                      int64
hospitals                  int64
shops                      int64
aid_center

In [9]:
for i in df.columns:
    print(i, df[i].unique())

id [    2     7     8 ..., 30263 30264 30265]
message ['Weather update - a cold front from Cuba that could pass over Haiti'
 'Is the Hurricane over or is it not over'
 'Looking for someone but no name' ...,
 "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families."
 'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.'
 'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.']
original [ 'Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de pluie isolee sont encore prevues sur notre region ce soi'
 'Cyclone nan fini osinon li pa fini'
 'Patnm, di Maryani relem pou li b

So there are two columns(Genre and realetd) with more than 2 classes.So we would use pd.dummies to change them.

In [10]:
X = df.message
Y = df[df.columns[5:]]
dummies = pd.get_dummies(df[['related','genre']])
Y = pd.concat([Y, dummies], axis=1)

### 2. Write a tokenization function to process your text data

In [11]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [12]:
#  From udacity's lesson

def tokenize(text):
    '''
    A fucntion to tokenizing a given text.
    
    Input  : text
    Output : A list of clean tokens 
    
    '''
    
    #Detct all urls
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
    # Split the text into tokens
    tokens = word_tokenize(text)
    
    # Chnage words into their basic form
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
# Create pipeline with Classifier
model = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('model', MultiOutputClassifier(RandomForestClassifier(class_weight="balanced")))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
# Split data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

# Train and Predict
model.fit(X_train.values, Y_train.values)
Y_pred = model.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
def classification_result(Y_test,Y_pred):
    '''
    A function to print classification report
    
    Input:  
          Y_test 
          Y_pred 
    
    Output: 
          Classification
    '''
    results = pd.DataFrame([[0, 0,0,0]],columns=['Message_type', 'f_score', 'precision', 'recall'])
    j = 0
    for i in Y_test.columns:
        prec = precision_score(Y_test[i], Y_pred[:,j],average='weighted')
        rec = recall_score(Y_test[i], Y_pred[:,j],average='weighted')
        f1  = f1_score(Y_test[i], Y_pred[:,j],average='weighted')
        model_results =  pd.DataFrame([[i, f1,prec,rec]],
                                      columns=['Message_type', 'f_score', 'precision', 'recall'])
        results = results.append(model_results, ignore_index = True)
    print('Weighted_average f_score:', results['f_score'].mean())
    print('Weighted_averageprecision:', results['precision'].mean())
    print('Weighted_average recall:', results['recall'].mean())
    return results[1:]

In [16]:
classification_result(Y_test,Y_pred)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Weighted_average f_score: 0.821102929065
Weighted_averageprecision: 0.855650788866
Weighted_average recall: 0.820399755874


,Message_type,f_score,precision,recall
1,request,0.866995,0.879069,0.884803
2,offer,0.953320,0.991746,0.917760
3,aid_related,0.566018,0.716592,0.647238
4,medical_help,0.859968,0.860157,0.859780
5,medical_products,0.896481,0.910299,0.883583
6,search_and_rescue,0.925242,0.950433,0.902655
7,security,0.935820,0.966648,0.907690
8,military,0.909932,0.931914,0.889075
9,child_alone,0.959270,1.000000,0.921727
10,water,0.901257,0.908316,0.894873


### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
parameters =  {
              'model__estimator__n_estimators': [30, 50], 
              'model__estimator__min_samples_split': [2, 5]} 

cv = GridSearchCV(model, param_grid=parameters)
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None,
            verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'model__estimator__n_estimators': [30, 50], 'model__estimator__min_samples_split': [2, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

# Train and Predict
cv.fit(X_train.values, Y_train.values)

Y_pred = cv.predict(X_test)

In [18]:
# The model is taking consideranble amount of time to execute so I am not going to use gridsearch 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
classification_result(Y_test,Y_pred)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Weighted_average f_score: 0.821102929065
Weighted_averageprecision: 0.855650788866
Weighted_average recall: 0.820399755874


,Message_type,f_score,precision,recall
1,request,0.866995,0.879069,0.884803
2,offer,0.953320,0.991746,0.917760
3,aid_related,0.566018,0.716592,0.647238
4,medical_help,0.859968,0.860157,0.859780
5,medical_products,0.896481,0.910299,0.883583
6,search_and_rescue,0.925242,0.950433,0.902655
7,security,0.935820,0.966648,0.907690
8,military,0.909932,0.931914,0.889075
9,child_alone,0.959270,1.000000,0.921727
10,water,0.901257,0.908316,0.894873


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [20]:
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.